In [5]:
def filter_data(path:str):
    '''
    returns:
    filtered_df: the edited dataframe  
    '''
    import pandas as pd
    import numpy as np
    df = pd.read_csv(path)

    # drop initials nans
    df = df.dropna(subset=['pl_orbper','sy_pnum','pl_controv_flag'])

    df = df.where(np.logical_and(df['pl_orbper']>.5,df['pl_orbper']<15))
    df = df.where(df['pl_controv_flag']==0)
    df = df.where(df['sy_pnum']==1)

    # filter out the data where the given fields did not meet the conditions
    filtered_df = df.dropna(subset=['pl_orbper','pl_controv_flag','sy_pnum'])

    return filtered_df

In [6]:

from sunnyhills.pipeline_functions import download_and_preprocess,run_bls
from sunnyhills.plotting import bls_validation_mosaic
def bls_check():
    df = filter_data('C:/Users/60002/Documents/GitHub/sunnyhills/personal_epochs/veronica/may/validation/tess nasa exo archive confirmed.csv')
    list = df.sample(n=10)

    for tid in list:
        download_and_preprocess(tid)
        


ModuleNotFoundError: No module named 'sunnyhills'

In [4]:
bls_check()